In [5]:
#Data Manipulation packages
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize 

#webscraping packages
from bs4 import BeautifulSoup as BS
import requests
import json
#import geocoder #Using the geocoder to find long and lat
#from geopy.geocoders import Nominatim

#Machine Learning Packages
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans # All packages related to Kmeans and the evaluation
from sklearn import metrics

#Visualization Packages
import matplotlib.pyplot as plt
!conda install -c conda-forge folium=0.5.0 --yes
import folium# Used for creating interactive maps
import plotly.plotly as py #preferred package to create interactive plots and maps
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#declaring this. Much similar to %matplotlib inline.
init_notebook_mode(connected=True)


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.90 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.78 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.84 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.66 MB/s


In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

postal =requests.get(url)
html = postal.text

soup = BS(html, 'lxml')

In [7]:
soup_pret = soup.prettify()

table = soup.find('table')

In [8]:
Post = []
Borough = []
Neighborhood = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        Post.append(cells[0].get_text())
        Borough.append(cells[1].get_text())
        Neighborhood.append(cells[2].get_text())

print(Post[:10])
print(Borough[:10])
print(Neighborhood[:10])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A', 'M8A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'North York', "Queen's Park", 'Not assigned']
['Not assigned\n', 'Not assigned\n', 'Parkwoods\n', 'Victoria Village\n', 'Harbourfront\n', 'Regent Park\n', 'Lawrence Heights\n', 'Lawrence Manor\n', 'Not assigned\n', 'Not assigned\n']


In [9]:
neigh = []

for i in Neighborhood:
    index = i.find("\n")
    neigh.append(i[:index].rstrip())

print(neigh[:10])

['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront', 'Regent Park', 'Lawrence Heights', 'Lawrence Manor', 'Not assigned', 'Not assigned']


In [10]:
df = pd.DataFrame(Post, columns = ['Postcode'])
df['Borough'] = Borough
df['Neighborhood'] = neigh

In [11]:
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop = True)

df1 = df1.copy()

df1.loc[6, 'Neighborhood'] = "Queen's Park"

df2 = pd.DataFrame(df1.groupby(['Postcode', 'Borough'])['Neighborhood'].unique())
df2.reset_index(inplace = True)

In [12]:
for i, k in df2['Neighborhood'].iteritems():
    l =list(k)
    t = ''
    for p, m in enumerate(l):
        t = t + str(l[p])+', '
    
    df2.loc[i,'Neighborhood'] = t[:-2]

In [13]:
geo_coords = pd.read_csv('Geospatial_Coordinates.csv')
df2 = pd.merge(df2, geo_coords, left_on = 'Postcode', right_on = 'Postal Code').drop('Postal Code', axis = 1)

FileNotFoundError: File b'Geospatial_Coordinates.csv' does not exist